In [36]:
import numpy as np
import pandas as pd
import gmaps
import requests
import json

# Google developer API key
from config import gkey

In [4]:
# read previous outputs into new dataframes
toronto_schools = pd.read_csv("output_data/toronto_schools.csv")
catholic_schools = pd.read_csv("output_data/toronto_catholic_schools.csv")

In [5]:
top_ts=toronto_schools.nlargest(10,"RATINGS")[["SCHOOL NAME","FSA","RATINGS"]]
top_cs=catholic_schools.nlargest(10,"RATINGS")[["SCHOOL NAME","FSA","RATINGS"]]

In [6]:
# Visualize top 10 Toronto Schools
top_ts

,SCHOOL NAME,FSA,RATINGS
11,Fleming,M1B,9.5
166,Whitney,M4T,9.5
129,Denlow,M3B,9.3
164,John Ross Robertson,M4R,9.3
245,Humber Valley Village,M9A,9.3
115,Beverly Glen,M1W,9.2
163,Blythwood,M4N,9.1
211,Ossington/Old Orchard,M6J,9.1
233,Swansea,M6S,9.1
88,Iroquois,M1S,9.0


In [7]:
# Visualize top 10 Catholic Schools
top_cs

,SCHOOL NAME,FSA,RATINGS
104,St Sebastian,M6H,10.0
84,St Michael's Choir,M5B,8.8
132,Josyf Cardinal Slipyj,M9B,8.8
133,St Clement,M9C,8.8
56,St Bonaventure,M3C,8.7
81,Our Lady of Perpetual Help,M4T,8.6
141,St Demetrius,M9P,8.4
35,Prince of Peace,M1V,8.3
40,St Henry,M1W,8.3
122,St Josaphat,M8V,8.3


In [8]:
# read output house prices into new dataframe
house_price=pd.read_csv("output_data/toronto_api_postal_combined and filtered with price range 100K to 10M.csv")
house_price.head()

,Index,AreaName,Price ($),lat,lng,Address,City,Postal Code,FSA
0,1,Downtown,100000,43.669567,-79.388596,#13 - 55 Bloor St,Toronto,M4W 1A6,M4W
1,2,Emery,102000,43.749187,-79.542702,#704 - 3390 Weston Rd,Toronto,M9M 2X3,M9M
2,3,Emery,104900,43.749187,-79.542702,#1104 - 3390 WESTON RD,Toronto,M9M 2X3,M9M
3,4,Mimico,105000,43.615341,-79.492996,170 THE QUEENS WAY,Toronto,M8V 2N9,M8V
4,5,Mimico,105000,43.615341,-79.492996,170 THE QUEENS WAY,Toronto,M8V 2N9,M8V


In [14]:
top_ts_list=list(top_ts["FSA"])
top_cs_list=list(top_cs["FSA"])

ts_house_price=house_price[house_price["FSA"].isin(top_ts_list)]
cs_house_price=house_price[house_price["FSA"].isin(top_cs_list)]

ts_house_price.head()

,Index,AreaName,Price ($),lat,lng,Address,City,Postal Code,FSA
6,7,Agincourt,105000,43.788311,-79.268082,#351 - 4438 SHEPPARD AVE E,Toronto,M1S 1V2,M1S
17,18,Agincourt,109988,43.788311,-79.268082,#238 - 4438 SHEPPARD AVE E,Toronto,M1S 1V2,M1S
18,19,Malvern,114900,43.803925,-79.216507,#105 - 8 Silverbell Grve,Toronto,M1B 2L7,M1B
25,26,Agincourt,118888,43.788311,-79.268082,#223 - 4438 SHEPPARD AVE E,Toronto,M1S 1V2,M1S
29,30,Little Italy,119000,43.652119,-79.407463,#3 rd Fl- 770-772 DUNDAS ST W,Toronto,M6J 1V1,M6J


In [15]:
cs_house_price.head()

,Index,AreaName,Price ($),lat,lng,Address,City,Postal Code,FSA
3,4,Mimico,105000,43.615341,-79.492996,170 THE QUEENS WAY,Toronto,M8V 2N9,M8V
4,5,Mimico,105000,43.615341,-79.492996,170 THE QUEENS WAY,Toronto,M8V 2N9,M8V
15,16,Armdale,109900,43.820740,-79.261734,#218 - 3300 Mcnicoll Ave,Toronto,M1V 5J6,M1V
26,27,Wallace Emerson,119000,43.667503,-79.442085,1245 Dupont Street,Toronto,M6H 2A6,M6H
27,28,Wallace Emerson,119000,43.667503,-79.442085,1245 DUPONT ST,Toronto,M6H 2A6,M6H


In [21]:
print(top_cs_list)
print(top_ts_list)

['M6H', 'M5B', 'M9B', 'M9C', 'M3C', 'M4T', 'M9P', 'M1V', 'M1W', 'M8V']
['M1B', 'M4T', 'M3B', 'M4R', 'M9A', 'M1W', 'M4N', 'M6J', 'M6S', 'M1S']


In [48]:
# geocoordinates
target_coordinates = "43.6532, 79.3832"
target_search = "Fleming Public School"
target_radius = 100000
# target_type = "school"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
#     "type": target_type,
    "key": gkey
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params).json()

In [49]:
response

{'html_attributions': [], 'results': [], 'status': 'ZERO_RESULTS'}